In [1]:
import numpy as np
import pandas as pd
import nbformat
import re
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import openai
from openai import RateLimitError, OpenAIError
from openpyxl import Workbook
from openpyxl.styles import Font
import os
from dotenv import load_dotenv
from pprint import pprint
from openai import OpenAI
from fpdf import FPDF
from tqdm import tqdm
import sys
from joblib import Parallel, delayed
import logging
import logging
import os
from joblib import Parallel, delayed
from tqdm import tqdm
from io import StringIO

In [2]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key is None:
    raise ValueError("API key is not set")

# Inicializar la API de OpenAI
openai.api_key = openai_api_key

In [3]:
import random

def generar_nombres_apellidos(num_nombres):
    nombres = [
    "Carlos", "María", "Javier", "Ana", "Luis", "Sofía", "Fernando", "Laura", "Pablo", "Marta",
    "Alberto", "Clara", "Diego", "Isabel", "Ricardo", "Elena", "Manuel", "Carmen", "Ignacio", "Teresa",
    "Francisco", "Beatriz", "Eduardo", "Lucía", "Antonio", "Patricia", "Miguel", "Silvia", "Raúl", "Cristina",
    "Alejandro", "Sandra", "Roberto", "Victoria", "Gabriel", "Inés", "Álvaro", "Paula", "Daniel", "Eva",
    "José", "Irene", "Adrián", "Nuria", "Ángel", "Mónica", "Jaime", "Gloria", "Jorge", "Rosa"
    ]

    apellidos = [
    "García", "Martínez", "López", "Sánchez", "González", "Pérez", "Rodríguez", "Fernández", "Gómez", "Díaz",
    "Hernández", "Álvarez", "Ruiz", "Jiménez", "Moreno", "Muñoz", "Romero", "Alonso", "Gutiérrez", "Navarro",
    "Torres", "Domínguez", "Vázquez", "Ramos", "Gil", "Serrano", "Blanco", "Molina", "Castro", "Ortiz",
    "Rubio", "Marín", "Sanz", "Núñez", "Iglesias", "Medina", "Garrido", "Cortés", "Castillo", "Santos",
    "Guerrero", "Ortega", "Delgado", "Prieto", "Vega", "Méndez", "Cabrera", "Fuentes", "León", "Herrera"
    ]

    
    lista_nombres_completos = []
    for _ in range(num_nombres):
        nombre_completo = f"{random.choice(apellidos)}_{random.choice(nombres)}"
        lista_nombres_completos.append(nombre_completo)
    
    return lista_nombres_completos


In [4]:
import nbformat
import logging

def extrae_enunciados_y_soluciones(examen_file):
    """
    Extrae los enunciados y las soluciones desde un notebook de examen.

    Parameters:
        examen_file (str): Ruta del archivo del notebook de examen.

    Returns:
        tuple: Dos listas, una con los enunciados y otra con las soluciones.
    """
    enunciados = []
    soluciones = []

    # Logger específico para esta función
    logger = logging.getLogger('extrae_enunciados_y_soluciones')
    log_stream = logging.StreamHandler()
    logger.addHandler(log_stream)
    logger.setLevel(logging.DEBUG)

    try:
        # Leer el notebook
        with open(examen_file, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
        logger.info(f"Notebook {examen_file} leído correctamente.")
    except FileNotFoundError:
        error_msg = f"El archivo {examen_file} no se encontró."
        logger.critical(error_msg)
        raise RuntimeError(error_msg)
    except Exception as e:
        error_msg = f"Error al leer el archivo {examen_file}: {e}"
        logger.critical(error_msg)
        raise RuntimeError(error_msg)

    try:
        # Procesar las celdas del notebook
        for cell in notebook.cells:
            if cell.cell_type == 'markdown':
                cell_content = cell['source'].strip()

                # Extraer Enunciado
                if cell_content.startswith("## Ejercicio"):
                    enunciado = cell_content.split("## Ejercicio")[1].strip().split("Criterios:")[0].strip()
                    enunciados.append(enunciado)

            elif cell.cell_type == 'code':
                # Extraer Solución
                solucion = cell['source'].strip()
                soluciones.append(solucion)

        logger.info("Enunciados y soluciones extraídos correctamente.")
    
    except Exception as e:
        error_msg = f"Error inesperado al procesar el archivo {examen_file}: {e}"
        logger.critical(error_msg)
        raise RuntimeError(error_msg)

    return enunciados, soluciones


In [5]:
def leer_tipos_errores(tipos_errores_file):
    """
    Lee el archivo que contiene la descripción de los tipos de errores y devuelve un diccionario con la información.

    Parameters:
        tipos_errores_file (str): Ruta al archivo de texto que contiene los tipos de errores.

    Returns:
        dict: Un diccionario donde las claves son los tipos de errores y los valores son diccionarios con
              las descripciones, ejemplos e instrucciones adicionales.
    """
    tipos_errores = {}
    
    with open(tipos_errores_file, 'r', encoding='utf-8') as file:
        contenido = file.read().split("---")  # Asume que los bloques están separados por ---
    
    for seccion in contenido:
        lineas = seccion.strip().split("\n")
        
        if len(lineas) < 3:  # Asegura que la sección tiene al menos tres líneas
            continue
        
        nombre_error = lineas[0].replace("### ", "").strip()  # Extrae el nombre del error
        
        # Extrae las secciones de descripción, ejemplo e instrucciones adicionales
        descripcion = ""
        ejemplo = ""
        instrucciones_adicionales = ""
        
        for linea in lineas[1:]:
            if linea.startswith("Descripción:"):
                descripcion = linea.replace("Descripción:", "").strip()
            elif linea.startswith("Ejemplo:"):
                ejemplo = linea.replace("Ejemplo:", "").strip()
            elif linea.startswith("Instrucciones Adicionales:"):
                instrucciones_adicionales = linea.replace("Instrucciones Adicionales:", "").strip()
        
        tipos_errores[nombre_error] = {
            'descripcion': descripcion,
            'ejemplo': ejemplo,
            'instrucciones_adicionales': instrucciones_adicionales
        }
    
    return tipos_errores



In [6]:
import re
import logging
from openai import OpenAI

def generar_solucion_con_error(solucion_correcta, tipo_error, descripcion_error, instrucciones_adicionales):
    """
    Genera una versión de la solución con un error específico.

    Parameters:
        solucion_correcta (str): El código correcto.
        tipo_error (str): El tipo de error que se desea introducir.
        descripcion_error (str): Descripción del error.

    Returns:
        str: Una versión del código con el error introducido.
    """
    # Crear un logger específico para esta función
    logger = logging.getLogger('generar_solucion_con_error')
    logger.setLevel(logging.DEBUG)
    
    # prompt = f"""Eres un asistente de programación. Tengo un código correcto y quiero que generes una versión de este código con un error.
    # Tipo de error: {tipo_error}
    # Descripción del error: {descripcion_error}
    
    # Código correcto:
    # {solucion_correcta}
    
    # Genera solo el código con el error:
    # """
    
    prompt = f"""
    Eres un asistente de programación altamente capacitado. Tengo un código que es completamente correcto y quiero que generes una versión de este código que contenga un error específico. 

    **Tipo de error**: {tipo_error}
    **Descripción del error**: {descripcion_error}

    A continuación, te proporciono el código correcto. Tu tarea es introducir el error indicado de manera sutil, asegurándote de que el código aún parezca plausible pero produzca el error descrito. Si ya has generado este tipo de error anteriormente, intenta variarlo de alguna manera (por ejemplo, cambia la ubicación del error, utiliza una sintaxis diferente, altera un parámetro distinto, etc.).

    **Código correcto**:
    ```python
    {solucion_correcta}

    Instrucciones específicas: {instrucciones_adicionales}

    Por favor, genera solo el código con el error ahora: """
    
    try:
        cliente = OpenAI()
        logger.debug("Cliente de OpenAI creado correctamente.")
        
        response = cliente.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a programming teaching assistant evaluating student code."},
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extraer el contenido del código y eliminar texto no deseado
        codigo_con_error = response.choices[0].message.content.strip()

        # Opcional: filtrar solo el bloque de código, si el modelo devuelve texto adicional.
        # Se asume que el código está dentro de un bloque de triple comillas, lo cual puede requerir ajustes según el comportamiento real.
        codigo_con_error = re.search(r'```python(.*?)```', codigo_con_error, re.DOTALL)
        if codigo_con_error:
            codigo_con_error = codigo_con_error.group(1).strip()

        logger.debug("Código con error generado correctamente.")
        
    except Exception as e:
        logger.error(f"Error al generar el código con error: {e}")
        codigo_con_error = solucion_correcta  # En caso de error, devolver la solución correcta sin modificar.

    return codigo_con_error



In [7]:
import nbformat as nbf

def generar_notebook(nombre_alumno, enunciados, soluciones, archivo_salida):
    """
    Genera un notebook Jupyter con las soluciones del alumno.

    Parameters:
        nombre_alumno (str): Nombre del alumno (se usará para personalizar el notebook).
        enunciados (list): Lista de enunciados de los ejercicios.
        soluciones (list): Lista de soluciones (código) correspondientes a los ejercicios.
        archivo_salida (str): Ruta donde se guardará el notebook generado.
    """
    # Crear una nueva lista de celdas
    celdas = []

    # Crear una celda de Markdown con el nombre del alumno
    celdas.append(nbf.v4.new_markdown_cell(f"# Examen de Finanzas - {nombre_alumno}"))

    # Añadir las celdas de enunciados y soluciones
    for enunciado, solucion in zip(enunciados, soluciones):
        # Celda de Markdown con el enunciado
        celdas.append(nbf.v4.new_markdown_cell(f"### Ejercicio\n{enunciado}"))
        # Celda de código con la solución
        celdas.append(nbf.v4.new_code_cell(solucion))

    # Crear el nuevo notebook con las celdas generadas
    nuevo_notebook = nbf.v4.new_notebook(cells=celdas)

    # Escribir el notebook en el archivo de salida
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        nbf.write(nuevo_notebook, f)

    print(f"Notebook {archivo_salida} generado correctamente.")


In [8]:
import nbformat as nbf
import os

def crear_notebook_para_alumno(nombre_alumno, enunciados, soluciones_alumno, output_dir):
    """
    Crea un notebook para un alumno específico con enunciados y soluciones.

    Parameters:
        nombre_alumno (str): Nombre del alumno para personalizar el notebook.
        enunciados (list): Lista de enunciados de los ejercicios.
        soluciones_alumno (list): Lista de soluciones (código) correspondientes a los ejercicios.
        output_dir (str): Directorio donde se guardará el notebook generado.
    """
    # Crear un nuevo notebook
    nb = nbf.v4.new_notebook()

    # Agregar las celdas con el contexto y los enunciados/soluciones
    cells = []
    
    # Asumimos que 'enunciados' y 'soluciones_alumno' son listas correspondientes
    for i, (enunciado, solucion) in enumerate(zip(enunciados, soluciones_alumno), start=1):
        # Agregar enunciado como celda de markdown con numeración
        cells.append(nbf.v4.new_markdown_cell(f"## Ejercicio {i}\n\n{enunciado}"))
        # Agregar solución como celda de código
        cells.append(nbf.v4.new_code_cell(solucion))
    
    # Añadir todas las celdas al notebook
    nb['cells'] = cells
    
    # Escribir el notebook a un archivo
    archivo_salida = os.path.join(output_dir, f"{nombre_alumno}.ipynb")
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        nbf.write(nb, f)

    print(f"Notebook {archivo_salida} generado correctamente.")


In [9]:
from joblib import Parallel, delayed

def generar_notebooks_examen(examen_file, tipos_errores_file, output_dir, num_alumnos, errores_output_file, prob_error=0.4, tipo_error_fijo=None):
    """
    Genera múltiples notebooks de examen con errores introducidos aleatoriamente o de un tipo específico, 
    y guarda un registro de los errores en un archivo Excel.

    Parameters:
        examen_file (str): Ruta del archivo del examen en formato notebook.
        tipos_errores_file (str): Ruta del archivo que contiene los tipos de errores posibles.
        output_dir (str): Directorio donde se guardarán los notebooks generados.
        num_alumnos (int): Número de notebooks de alumnos a generar.
        errores_output_file (str): Ruta del archivo Excel donde se guardará el registro de errores.
        prob_error (float): Probabilidad de que se introduzca un error en cada ejercicio (por defecto 0.4).
        tipo_error_fijo (str, opcional): Tipo de error a introducir en los ejercicios. Si es None, se eligen errores aleatorios.
    """
    # Extraer enunciados y soluciones correctas
    enunciados, soluciones_correctas = extrae_enunciados_y_soluciones(examen_file)
    
    # Leer tipos de errores desde el archivo
    tipos_errores = leer_tipos_errores(tipos_errores_file)
    
    # Generar nombres de alumnos
    nombres_alumnos = generar_nombres_apellidos(num_alumnos)
    
    # Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Estructura para almacenar los errores por alumno
    errores_registro = []

    # Función para procesar cada alumno
    def procesar_alumno(nombre_alumno):
        soluciones_alumno = []
        for i, solucion_correcta in enumerate(soluciones_correctas):
            if random.random() <= prob_error:  # Evaluar la probabilidad de introducir un error
                if tipo_error_fijo and tipo_error_fijo in tipos_errores:
                    tipo_error = tipo_error_fijo
                else:
                    tipo_error = random.choice(list(tipos_errores.keys()))
                
                descripcion_error = tipos_errores[tipo_error]['descripcion']
                instrucciones_adicionales = tipos_errores[tipo_error].get('instrucciones_adicionales', '')
                solucion_con_error = generar_solucion_con_error(solucion_correcta, tipo_error, descripcion_error, instrucciones_adicionales)
                soluciones_alumno.append(solucion_con_error)
                
                # Registrar el error
                errores_registro.append({
                    "Alumno": nombre_alumno,
                    "Ejercicio": i + 1,
                    "Tipo de Error": tipo_error,
                    "Descripción": descripcion_error
                })
            else:
                soluciones_alumno.append(solucion_correcta)
        
        # Crear el notebook para este alumno
        crear_notebook_para_alumno(
            nombre_alumno, enunciados, soluciones_alumno, output_dir
        )

    # Ejecutar la creación de notebooks en paralelo con verbose=13
    Parallel(n_jobs=-1, verbose=13)(
        delayed(procesar_alumno)(nombre_alumno) for nombre_alumno in nombres_alumnos
    )

    # Guardar el registro de errores en un archivo Excel
    df_errores = pd.DataFrame(errores_registro)
    df_errores.to_excel(errores_output_file, index=False)
    print(f"Registro de errores guardado en {errores_output_file}")


In [10]:
# Ejemplo de uso
examen_file = "/workspace/examenes/examen_finanzas_gpt.ipynb"
tipos_errores_file = "/workspace/tipos_errores.txt"
output_dir = "/workspace/entregas"
errores_output_file = "/workspace/reports/errores.xlsx"
num_alumnos = 10
prob_error = 0.99
tipo_error="Errores Lógicos"

generar_notebooks_examen(examen_file, tipos_errores_file, output_dir, num_alumnos, errores_output_file, prob_error, tipo_error_fijo=tipo_error)

Notebook /workspace/examenes/examen_finanzas_gpt.ipynb leído correctamente.
Enunciados y soluciones extraídos correctamente.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.


Notebook /workspace/entregas/Ortiz_Diego.ipynb generado correctamente.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   19.1s


Notebook /workspace/entregas/Navarro_Elena.ipynb generado correctamente.


[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:   20.2s remaining:  1.3min


...

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   22.7s remaining:   15.2s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   22.9s remaining:    9.8s


Notebook /workspace/entregas/Molina_Luis.ipynb generado correctamente.
Notebook /workspace/entregas/Jiménez_Luis.ipynb generado correctamente.


[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:   23.7s remaining:    5.9s


Notebook /workspace/entregas/Rodríguez_Ángel.ipynb generado correctamente.
Registro de errores guardado en /workspace/reports/errores.xlsx


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.4s finished
